# Домашнее задание №6. Ансамбли. Случайный лес. Градиентный бустинг

## Ф.И.О: Маслак Иван Александрович

### Описание.

Домашнее задание состоит из **2**-х частей:
  - реализация модулей:
    - bagger
    - booster
    - sampler
-------------------
 * Реализации можно частично проверить через юнит-тесты (запускаются командой ``pytest tests``).
-------------------
  - экспериментальная часть

**На проверку требуется отправить zip архив, который будет содержать следующие файлы:**
  - модуль ``ensemble`` с реализованными модулями(bagger, booster, sampler)
 - заполненный блокнот в формате ``.ipynb``
  - заполненный блокнот в формате ``.html`` **(в jupyter: File -> Save and Export Notebook As -> HTML -> ...)**

## 1.1 Сэмплирование случайных объектов и признаков (2 points)

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

`BaseSampler` класс, который будет упрощать семплирование различных подмассивов данных. 
Реализуйте метод:
  - `sample_indices`, который по числу сущностей `n_objects` возращает случайную подвыборку индексов.

Используйте атрибуты:
  - `self.random_state`, чтобы результаты семпплирования воспроиводились
  - `self.bootstrap`, если нужно выбрать случайную подвыборку с возвращением.

У класса `ObjectSampler` реализован метод:
  - `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них.

В классе `FeaturesSampler` реализован метод:
- `sample`, который возвращает случайную подвыборку признаков для объектов.

## 1.2. Бэггинг (5 points)

Суть бэггинга заключается в обучении нескольких "слабых" базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов.

В данном задании реализация **случайного леса будет бэггингом над решающими деревьями**. 

Реализация случайного леса представлена в классе `RandomForestClassifier`.

## 1.3. Градиентный бустинг (7 points)

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. 

Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. У каждого следующего алгоритма вычисляется ___вес___ $\omega$, с которым он входит в ансамбль. 

Также есть параметр скорости обучения (___learning rate___ - $lr$), который не позволяет алгоритму переобучитсья. 

Вес $\omega$ можно находить, используя одномерную оптимизацию. 

Рассмотрим процедуру обучения по шагам (будем рассматривать случай бинарной классификации c метками классов {0,1}, чтобы не усложнять жизнь):
1. Настройка базового алгоритма $b_0$ (в данном случае это решающее дерево).
    
    $$\text{Алгоритм настраиваются на $y$ с помощью функции MSE.}$$
    
2. Будем обозначать текущий небазовый алгоритм - $a$:
    
    $$a_i(x) = \sum_{j=0}^i \omega_j b_j(x) $$
    
3. Настройка базового алгоритма $b_i$ (обычно это регрессионное дерево):
    
    $$b_i = \arg \min_b \sum_{j=1}^l (b(x_j) + \nabla L(a_{i-1}(x_j), y))^2,$$
    т.е. выход очередного базового алгоритма подстраивается под антиградиент функции потерь
    
4. Настройка веса базового алгоритма $\omega_i$:
    
    $$\omega_i = \min_{\omega > 0} \sum_{j=1}^l L(a_{i-1} + \omega b_i(x_j), y) $$
    
Для задачи классфикации будем использовать логистическую функцию потерь. Немного упростим ее:

$$L = -y\log\sigma(a) - (1-y)\log(1 - \sigma(a)) = -\log(1 - \sigma(a)) - y \log \frac{\sigma(a)}{1 - \sigma(a)},$$
где $\sigma$ - функция сигмоиды. 

Ответ после очередного базового алгоритма надо прогонять через сигмоиду, т.к. не гарантируется, что ответы будут лежать на [0,1] - в этом особенность базового алгоритма (который является регрессионным).

Преобразуем:
$$\log (1 - \sigma(a)) = \log \frac{1}{1 + \exp(a)} = -\log(1 + \exp(a)) $$

$$\log (\frac{\sigma(a)}{1 - \sigma(a)}) = \log(\exp(a)) = a $$
 
Таким образом:

$$L = -ya + \log(1 + \exp(a))$$

Тогда будем вычислять градиент как:
 
$$\nabla L = - y + \sigma(a)$$

В классе `Booster` реализуйте методы:
* `_fit_first_estimator` – построение первой модели (первого приближения данных);
* `fit` – обучение алгоритма градиентного бустинга (обучение первой и последующих базовых моделей);
* `predict` – получение предсказаний алгоритма градиентного бустинга.

В классе `GradientBoostingClassifier` реализуйте методы:
* `_fit_base_estimator` - обучение очередной базовой модели;
* `_gradient` - расчет градиента функции ошибки;
* `_loss` - расчет функции ошибки (для одномерно оптимизации).

## 2. Эксперименты (6 points)

Скачайте датасейт для экспериментов: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Колонка с ответами - RainTommorow.

In [21]:
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
%load_ext autoreload
%autoreload 2

from ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
import pandas as pd

In [33]:
data = pd.read_csv('weatherAUS.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Выделите признаки год/месяц/день:

In [34]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['day'] = data['Date'].dt.day

Посмотрим какие года есть в выборке:

In [35]:
data['year'].value_counts()

year
2016    17934
2014    17885
2015    17885
2009    16789
2010    16782
2013    16415
2012    15409
2011    15407
2017     8623
2008     2270
2007       61
Name: count, dtype: int64

Разделите выборку на три части (train, val и test) по временному принципу:
    
* train - 2007-2014
* val - 2015
* test - 2016-2017

In [36]:
indexes = {
    'train': data['year'].between(2007, 2014),
    'val': data['year'] == 2015,
    'test': data['year'].between(2016, 2017),
}

Здесь вы можете делать всевозможные преобразования признаков. 

Для того, чтобы получить качество, необходимое для преодоления бейзлайна, вам достаточно закодировать все категориальные признаки с помощью `LabelEncoder`, а также разумно обработать пропущенные значения.

In [37]:
data.drop(['Date'], axis=1, inplace=True)

In [38]:
categorical_features = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 
                        'RainToday', 'RainTomorrow']

for col in categorical_features:
    if col in data.columns:
        le = LabelEncoder()
        data[col] = data[col].fillna('Unknown')
        data[col] = le.fit_transform(data[col])
numerical_features = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
                      'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 
                      'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm',
                      'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm',
                      'year', 'month', 'day']

for col in numerical_features:
    if col in data.columns:
        if data[col].isnull().any():
            median = data[col].median()
            data[col] = data[col].fillna(median)


Ваш таргет - **RainTommorow**. Удалите его из обучающих данных, также удалите признак RISK_MM.

In [39]:
target_data = data['RainTomorrow']
data.drop(['RainTomorrow'], axis=1, inplace=True)

In [40]:
X_train, y_train = data[indexes['train']].values, target_data[indexes['train']].values
X_val, y_val = data[indexes['val']].values, target_data[indexes['val']].values
X_test, y_test = data[indexes['test']].values, target_data[indexes['test']].values

In [41]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,month,day
0,2,13.4,22.9,0.6,4.8,8.4,14,44.0,14,15,...,1007.7,1007.1,8.0,5.0,16.9,21.8,0,2008,12,1
1,2,7.4,25.1,0.0,4.8,8.4,15,44.0,6,16,...,1010.6,1007.8,5.0,5.0,17.2,24.3,0,2008,12,2
2,2,12.9,25.7,0.0,4.8,8.4,16,46.0,14,16,...,1007.6,1008.7,5.0,2.0,21.0,23.2,0,2008,12,3
3,2,9.2,28.0,0.0,4.8,8.4,4,24.0,9,0,...,1017.6,1012.8,5.0,5.0,18.1,26.5,0,2008,12,4
4,2,17.5,32.3,1.0,4.8,8.4,14,41.0,1,7,...,1010.8,1006.0,7.0,8.0,17.8,29.7,0,2008,12,5


Для каждого из алгоритмов достигнутое качество (**accuracy**) должно быть: 
* ***RandomForest > 0.84 (2 points)***
* ***GradientBoosting > 0.845 (2 points)***
* ***Отчет об экспериментах (2 points)***

Обучите каждый из алгоритмов до нужного качества, используйте валидационную выборку, чтобы подбирать гиперпараметры. Получите качество (accuracy) выше необходимого и на validation, и на test.

**Подсказка:** для визуализации прогресса обучения можно использовать бибилиотеку [`tqdm`](https://tqdm.github.io/).

In [46]:
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

#### Сначала обучим Random Forest

In [42]:
estimators_range = [50, 100, 150, 200, 250]
accuracies = []

print("Подбор n_estimators:")
for n in tqdm(estimators_range):
    rf = RandomForestClassifier(
        n_estimators=n,
        max_depth=None,
        min_samples_leaf=2,
        max_features_samples=0.8,
        max_objects_samples=0.8,
        random_state=42
    )
    rf.fit(X_train, y_train)
    y_val_pred = rf.predict(X_val)
    acc = accuracy_score(y_val, y_val_pred)
    accuracies.append(acc)
    print(f"  n_estimators={n}: accuracy={acc:.4f}")

best_n = estimators_range[np.argmax(accuracies)]
print(f"Лучшее n_estimators: {best_n}")


Подбор n_estimators:


  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [01:21<05:24, 81.06s/it]

  n_estimators=50: accuracy=0.8235


 40%|████      | 2/5 [03:39<05:45, 115.10s/it]

...

  n_estimators=150: accuracy=0.8257


 80%|████████  | 4/5 [11:43<03:24, 204.21s/it]

  n_estimators=200: accuracy=0.8261


100%|██████████| 5/5 [17:09<00:00, 205.97s/it]

  n_estimators=250: accuracy=0.8256
Лучшее n_estimators: 200


In [43]:
depth_range = [5, 10, 15, 20, 25, None]
accuracies = []

print("\nПодбор max_depth:")
for depth in tqdm(depth_range):
    rf = RandomForestClassifier(
        n_estimators=best_n,
        max_depth=depth,
        min_samples_leaf=3,
        max_features_samples=0.8,
        max_objects_samples=0.8,
        random_state=42
    )
    rf.fit(X_train, y_train)
    y_val_pred = rf.predict(X_val)
    acc = accuracy_score(y_val, y_val_pred)
    accuracies.append(acc)
    print(f"  max_depth={depth}: accuracy={acc:.4f}")

best_depth = depth_range[np.argmax(accuracies)]
print(f"Лучший max_depth: {best_depth}")



Подбор max_depth:


 17%|█▋        | 1/6 [01:38<08:10, 98.04s/it]

  max_depth=5: accuracy=0.8117


 33%|███▎      | 2/6 [04:19<09:01, 135.36s/it]

  max_depth=10: accuracy=0.8225


...

  max_depth=20: accuracy=0.8258


 83%|████████▎ | 5/6 [14:34<03:08, 188.97s/it]

  max_depth=25: accuracy=0.8260


100%|██████████| 6/6 [17:59<00:00, 179.91s/it]

  max_depth=None: accuracy=0.8253
Лучший max_depth: 25


In [44]:
leaf_range = [1, 2, 3, 5, 10]
accuracies = []

print("\nПодбор min_samples_leaf:")
for leaf in tqdm(leaf_range):
    rf = RandomForestClassifier(
        n_estimators=best_n,
        max_depth=best_depth,
        min_samples_leaf=leaf,
        max_features_samples=0.8,
        max_objects_samples=0.8,
        random_state=42
    )
    rf.fit(X_train, y_train)
    y_val_pred = rf.predict(X_val)
    acc = accuracy_score(y_val, y_val_pred)
    accuracies.append(acc)
    print(f"  min_samples_leaf={leaf}: accuracy={acc:.4f}")

best_leaf = leaf_range[np.argmax(accuracies)]
print(f"Лучший min_samples_leaf: {best_leaf}")



Подбор min_samples_leaf:


 20%|██        | 1/5 [03:29<13:57, 209.32s/it]

  min_samples_leaf=1: accuracy=0.8266


 40%|████      | 2/5 [06:51<10:15, 205.30s/it]

  min_samples_leaf=2: accuracy=0.8268


...

  min_samples_leaf=3: accuracy=0.8260


 80%|████████  | 4/5 [13:24<03:19, 199.01s/it]

  min_samples_leaf=5: accuracy=0.8272


100%|██████████| 5/5 [16:27<00:00, 197.54s/it]

  min_samples_leaf=10: accuracy=0.8289
Лучший min_samples_leaf: 10


In [45]:
rf_best = RandomForestClassifier(
    n_estimators=best_n,
    max_depth=best_depth,
    min_samples_leaf=best_leaf,
    max_features_samples=0.8,
    max_objects_samples=0.8,
    random_state=42
)

rf_best.fit(X_train, y_train)

y_val_pred = rf_best.predict(X_val)
y_test_pred = rf_best.predict(X_test)

val_acc = accuracy_score(y_val, y_val_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

Validation accuracy: 0.8285
Test accuracy: 0.8257


In [48]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

rf_balanced = RandomForestClassifier(
    n_estimators=best_n,
    max_depth=best_depth,
    min_samples_leaf=best_leaf,
    max_features_samples=0.8,
    max_objects_samples=0.8,
    random_state=42,
    **{'class_weight': {0: class_weights[0], 1: class_weights[1]}}
)

rf_balanced.fit(X_train, y_train)
y_test_pred_balanced = rf_balanced.predict(X_test)
test_acc_balanced = accuracy_score(y_test, y_test_pred_balanced)
print(f"Test accuracy с балансировкой классов: {test_acc_balanced:.4f}")

Test accuracy с балансировкой классов: 0.8281


## 3. Бонус. [AdaBoost](https://ru.wikipedia.org/wiki/AdaBoost) (5 points)

В алгоритме AdaBoost всем объектам обучения присваивается вес `weight`, который определяет степень важности объекта при обучении. И если текущая модель ошибается на некотором объекте, его вес увеличивается, и этот объект будет больше влиять на обучение следующей модели. 

Также, так как модели обучаются последовательно, они не равносильны между собой, поэтому у каждой модели тоже есть вес `alpha`, который определяет вес модели при суммировании ответов и зависит от количества ошибок `err` модели. На каждой итерации обучения, эти веса пересчитываются по формулам:

$$\alpha_j = \log\left(\frac{1-err_j}{err_j}\right),$$
где $err_j$ - ошибка классификации

$$w_{new}^t = \frac{w_{old}^{t}*\exp(-\alpha_j \cdot y(x^t) \cdot b_j(x^t))}{\sum\limits_{i=1}^m w_{old}^{t}*\exp(-\alpha_j \cdot y(x^i) \cdot b_j(x^i))}$$

Изначально все веса объектов $w^i$ равны (и нормированы на 1).

Вам предлагается полностью реализовать AdaBoost. Вы можете использовать предыдущие шаблоны, но учтите некоторые пункты:
* надо работать с метками {-1,1} - это обусловлено использованием экспоненциальной функции потерь
* метод `predict` представляет собой функцию сигмоид, примененную к сумме предсказаний всех моделей

Для реализации AdaBoost на датасете, полученном в предыдущем пункте требуется добиться точноcти:
* AdaBoost > 0.83

Полезные ссылки:

- [статья](https://face-rec.org/algorithms/Boosting-Ensemble/decision-theoretic_generalization.pdf)
- [nerc](https://neerc.ifmo.ru/wiki/index.php?title=%D0%91%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3,_AdaBoost)
- [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)